In [6]:
from langchain_openai import OpenAI
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import YoutubeLoader
import streamlit as st
import os

In [22]:

OPENAI_API_KEY = st.secrets["OPENAI_API_KEY"]
youtube_url = "https://www.youtube.com/watch?v=P1iCMyuwOfg&ab_channel=ChefReactions"
openai_embed_model = "text-embedding-ada-002"

In [7]:
loader = YoutubeLoader.from_youtube_url(youtube_url)

In [13]:
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)
embedding = OpenAIEmbeddings(api_key=OPENAI_API_KEY, model = "text-embedding-ada-002")
vector_store = FAISS.from_documents(chunks, embedding)

In [19]:
query1 = 'give me a bulleted list of the talking points'
query2 = 'please summarize the video'

In [16]:
query_answer = vector_store.similarity_search(query1)

In [21]:
print(query_answer[1].page_content)

which he did fota is a great way to start any Italian meal that was a 10 out of 10 followed that up with some BC spot prawns as well as a lobster salad very fresh very tasty 10 out of 10 actually you know what this whole meal was a 10 out of 10 so let's forget about the ratings going forward just assume that it's a 10 out of 10 the two pasta dishes this busiate with Bata was delicious rot noi fan tastic can't beat it I love a good mushroom pasta both of them guess what 10 out of 10 if you've watched my videos you've noticed that I do this thing where when I bite into something if I like it I nod like a idiot uh so that's the chef reaction seal of approval this grilled pork chop was incredibly balanced had a really nice acidity to it uh say that five times fast the kale salad was also good even though I'm not a rabbit we were then served a trio of desserts including this cake which was my personal favorite this cheesecake which was the cherries and fennel and this pistachio semi Fredo I

In [24]:
retriever = vector_store.as_retriever()
qa = RetrievalQA.from_chain_type(llm = OpenAI(), chain_type='stuff', retriever=retriever)


In [29]:
retriever_query = 'give me a bulleted list of the talking points'
results = qa.invoke(retriever_query)


In [34]:
print(results['result'])


- Introduction: name, Chef Reactions, Montreal
- Orange Julip: trademark drink, 7/10 rating
- Montreal spots: usually has a list, tried new places this time
- Guard M: Crouats (okay, dense), Octopus dish (10/10), tomato salad (6.5/10), pasta dish (10/10), cream puffs (10/10)
- Old Port of Montreal: pedestrian traffic only, European vibe, spring/summer live music, Bixie bikes, Old Montreal founded in 1642, touristy spot
- Montreal winter: sucks, underground city, two seasons (winter and construction)
- Tip for out of towners: don't drive, use Metro, $11 for day pass
- Eating Center: Timeout Market, fancy food court, $10 American for most expensive pasta dish, Cream puffs (10/10)
- Montreal Plaza: #21 on Canada's top 100 list, featured on Somebody Feed Phil, tasting menu, constantly changing, Potato foam (nostalgic), Sea urchin and cream with togarashi asparagus and dong, Spinach and scallop salad, Strawberries and asparagus dish, Turbo tempura salad, Signature dish (


In [ ]:
library.save_local("faiss_index")